In [2]:
# !pip install https://github.com/rikab/particleloader.git
from particleloader import load
from rikabplotlib.plot_utils import newplot, hist_with_outline

import numpy as np

ImportError: cannot import name 'newplot' from 'rikabplotlib' (unknown location)

In [3]:
cache_dir = "~./ParticleLoader"
lep_dijets = load("SPECTER_ee_dijets", 100000, cache_dir=cache_dir)

In [ ]:
from pyjet import cluster

names = ["hemisphere_1", "hemisphere_2"]

def cluster_hemispheres(events):

    jets = []
    particles0 = np.zeros((events.shape[0], 100, 4))
    particles1 = np.zeros((events.shape[0], 100, 4))
    particles = [particles0, particles1]

    for (e, event) in enumerate(events):

        print(e)

        # Set up 4-vectors
        four_vectors = []
        for particle in event:

            # Important that this is a list () and not a tuple []
            four_vectors.append((particle[0], particle[1], particle[2], particle[3]))
            
        four_vectors = np.array(four_vectors, dtype=[("E", "f8"), ("px", "f8"), ("py", "f8"), ("pz", "f8")])

        # Cluster with kT (p = 1)

        sequence = cluster(four_vectors, R=1, p=1, recomb_scheme = "WTA_modp_scheme", ep=True)
        subjets = sequence.exclusive_jets(2)

        for i in range(2):

            constituents = subjets[i].constituents_array(ep=True)
            num_constituents = constituents.shape[0]
            particles[i][e, :num_constituents] = [(c[0], c[1], c[2], c[3]) for c in constituents]


    return particles

particles = cluster_hemispheres(lep_dijets)
np.save("Data/LEP_ee_dijets_hemisphere_1.npy", particles[0])
np.save("Data/LEP_ee_dijets_hemisphere_2.npy", particles[1])